<a href="https://colab.research.google.com/github/davidrconejo/COA-Validation/blob/main/Verificacion_Semanal_COAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Paso 1. Instala librerías
!apt-get install -y poppler-utils
!pip install pytesseract pdf2image openpyxl

# Paso 2: Importar librerías
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
from google.colab import files
from PIL import Image

# Paso 3: Subir el archivo PDF escaneado
print("Sube el archivo PDF escaneado:")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Paso 4: Subir la lista maestra
print("Sube el archivo de la lista maestra:")
uploaded_master = files.upload()
master_path = list(uploaded_master.keys())[0]

# Paso 5: Programa - No modificar
df_maestra = pd.read_excel(master_path)
df_maestra.dropna(subset=['Fabricante', 'Lugar de Manufactura'], inplace=True)
df_maestra['Fabricante'] = df_maestra['Fabricante'].str.lower().str.strip()
df_maestra['Lugar de Manufactura'] = df_maestra['Lugar de Manufactura'].str.lower().str.strip()

proveedores = df_maestra['Fabricante'].unique().tolist()
lugares = df_maestra['Lugar de Manufactura'].unique().tolist()

# Paso 6: Convertir el PDF a imágenes
print("Convirtiendo PDF a imágenes")
images = convert_from_path(pdf_path)

# Paso 6: Analizar cada página del PDF
results = []
for i, img in enumerate(images):
    text = pytesseract.image_to_string(img).lower()

print("Buscando provedores y lugares de manufactura")

    # Buscar los proveedores y lugares detectados en el texto
    proveedores_detectados = [p for p in proveedores if p in text]
    lugares_detectados = [l for l in lugares if l in text]

    # Verificar si alguna combinación proveedor-lugar es válida
    combinaciones_validas = [
        (prov, lugar)
        for prov in proveedores_detectados
        for lugar in lugares_detectados
        if ((df_maestra['Fabricante'] == prov) & (df_maestra['Lugar de Manufactura'] == lugar)).any()
    ]

    # Resultado por página
    results.append({
        'Página': i + 1,
        'Proveedores Detectados': ', '.join(proveedores_detectados) or 'No encontrado',
        'Lugares Detectados': ', '.join(lugares_detectados) or 'No encontrado',
        'Combinacion Válida': len(combinaciones_validas) > 0,
        'Lista Combinaciones Validas': ', '.join([f'{p} + {l}' for p, l in combinaciones_validas]) or 'Ninguna'
    })

# Paso 7: Exportar resultados a Excel
df_resultados = pd.DataFrame(results)
df_resultados.to_excel("Verificacion Semanal de COAs.xlsx", index=False)

print("Verificacion completada.")
files.download("Verificacion Semanal de COAs.xlsx")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
📤 Sube el archivo PDF escaneado:


Saving Prueba5.pdf to Prueba5 (5).pdf
📤 Sube el archivo de la lista maestra (Excel con columnas 'Fabricante' y 'Lugar de Manufactura'):


Saving Lista Maestra.xlsx to Lista Maestra (5).xlsx
🖼️ Convirtiendo PDF a imágenes...
✅ Validación completada.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>